In [11]:
#!/usr/bin/env python3
"""
validate_associations_parallel_improved.py

Parallel validation of seismic detection associations with periodic checkpointing.
Includes enhanced error model with:
- Clock drift error term
- Picking error model
- Distance-based weighting
- Ray path deviation consideration
- More robust weighted least squares implementation
"""

import os
import numpy as np
import pandas as pd
from datetime import timedelta
from scipy.optimize import least_squares
from pyproj import Geod
from joblib import Parallel, delayed
from src.utils.data_reading.sound_data.sound_file_manager import DatFilesManager
from utils.physics.sound_model import ISAS_grid as isg

# === CONFIGURATION ===
ASSO_FILE = "F:/Association/2018/grids/2018/s_-60-5,35-120,350,0.8,0.6.npy"
OUTPUT_DIR = "F:/Association/validated"
OUTPUT_BASENAME = "s_-60-5,35-120,350,0.8,0.6"
BATCH_SIZE = 500            # checkpoint every N dates
N_JOBS = os.cpu_count() - 1  # leave one core free
GRID_LAT_BOUNDS = [-60, 5]
GRID_LON_BOUNDS = [35, 120]
GRID_SIZE = 350
DEPTH = 1250               # meters
SOUND_SPEED = 1480         # m/s, rough constant
GTOL = 1e-4
XTOL = 1e-4
ISAS_PATH = "F:/ISAS/86442/field/2018"

# Enhanced error model parameters
PICKING_ERROR_BASE = 2      # Base picking error in seconds
CLOCK_DRIFT_RATE = 1e-6         # Clock drift rate (ppm)
DIST_WEIGHT_FACTOR = 0.8        # How much distance affects weighting (0-1)
PATH_DEVIATION_FACTOR = 0.02    # How much potential ray path deviation increases with distance

# Precompute grid lat/lon
PTS_LAT = np.linspace(*GRID_LAT_BOUNDS, GRID_SIZE)
PTS_LON = np.linspace(*GRID_LON_BOUNDS, GRID_SIZE)

# Geod instance
geod = Geod(ellps="WGS84")

def grid_index_to_coord(indices):
    i, j = indices
    return [PTS_LAT[i], PTS_LON[j]]

def compute_enhanced_travel_time(lat, lon, station_lat, station_lon, b=0.0):
    """Enhanced travel time calculation with improved error model:

    Model: TT = (distance/sound_speed) + b

    Parameters:
    - lat, lon: source coordinates
    - station_lat, station_lon: station coordinates
    - b: constant bias in seconds (default 0.0), represents systematic timing errors

    Returns:
    - tt: estimated travel time in seconds
    - err: estimated error in travel time in seconds
    - dist_m: distance in meters
    """
    _, _, dist_m = geod.inv(lon, lat, station_lon, station_lat)

    # Base travel time calculation using constant sound speed
    base_tt = dist_m / SOUND_SPEED

    # Apply bias parameter
    tt = base_tt + b

    # Error modeling with multiple components
    picking_err = PICKING_ERROR_BASE  # Base error in picking arrival times
    deviation_err = PATH_DEVIATION_FACTOR * (dist_m/1000)  # Error increases with distance due to path complexity

    # Combined error (add in quadrature for independent error sources)
    total_err = np.sqrt(picking_err**2 + deviation_err**2)

    return tt, total_err, dist_m

def find_closest_station_index(src_lat, src_lon, detection_secs):
    distances = [
        geod.inv(src_lon, src_lat, station_lon, station_lat)[2]
        for (station_lat, station_lon), _ in detection_secs
    ]
    return np.argmin(distances)


def process_date(date, associations_list):
    """Process one date (cluster): refine source location & origin time with improved error model."""
    month = pd.to_datetime(date).month
    validated = []

    # Create simplified associations list to avoid serialization issues
    simplified_associations = []
    for detections, valid_points in associations_list:
        simple_detections = []
        for station_obj, det_time in detections:
            # Extract only necessary data from station_obj
            lat, lon = station_obj.get_pos()
            simple_detections.append(((lat, lon), det_time))
        simplified_associations.append((simple_detections, valid_points))

    for detections, valid_points in simplified_associations:
        # Skip tiny clusters
        if len(detections) < 7:
            continue

        # Build refined detections & station positions
        station_positions = [pos for pos, _ in detections]
        detection_times = [t for _, t in detections]
        ref_time = min(detection_times)
        detection_secs = [(pos, (t - ref_time).total_seconds()) for pos, t in detections]

        # Initial guess & bounds from valid_points
        if len(valid_points) > 0:
            coords = np.array([grid_index_to_coord(tuple(p)) for p in valid_points])
            lat0, lon0 = coords.mean(axis=0)
            margin = 1
            lat_min, lat_max = coords[:,0].min()-margin, coords[:,0].max()+margin
            lon_min, lon_max = coords[:,1].min()-margin, coords[:,1].max()+margin
        else:
            arr = np.array(station_positions)
            lat0, lon0 = arr[:,0].mean(), arr[:,1].mean()
            lat_min, lat_max = arr[:,0].min()-0.5, arr[:,0].max()+0.5
            lon_min, lon_max = arr[:,1].min()-0.5, arr[:,1].max()+0.5

        # Nearest station gives t0 guess
        dists = np.array([geod.inv(lon0, lat0, lat, lon)[2] for lat, lon in station_positions])
        nearest_idx = dists.argmin()
        t0_offset_guess = - (dists[nearest_idx] / SOUND_SPEED)

        # Parameter definitions:
        # lat, lon: source location coordinates
        # origin_time_offset: time offset from ref_time to the origin time (in seconds)
        # b: constant time bias for error model (in seconds)

        # Initial parameter guess
        x0 = [lat0, lon0, t0_offset_guess, 0.0]

        # Parameter bounds
        bounds = (
            [lat_min, lon_min, t0_offset_guess-1200, -60],  # Lower bounds
            [lat_max, lon_max, t0_offset_guess+1200, 60]     # Upper bounds
        )

        # Enhanced residual function with improved error model and weighting
        def residual(params):
            lat, lon, origin_time_offset, b = params
            errs = []
            sigmas = []

            for idx, (station_pos, sec) in enumerate(detection_secs):
                slat, slon = station_pos
                # Calculate travel time from source to station
                travel_time, terr, dist = compute_enhanced_travel_time(lat, lon, slat, slon, b=b)

                # Predicted arrival time = origin time + travel time
                predicted_arrival = origin_time_offset + travel_time

                # Residual = observed - predicted (in seconds)
                raw_err = sec - predicted_arrival

                sigma = terr + 0.1  # Add constant to avoid division by very small errors
                errs.append(raw_err)
                sigmas.append(sigma)

            return np.array(errs) / np.array(sigmas)

        # Solve with robust method
        try:
            res = least_squares(
                residual, x0, bounds=bounds,
                method='trf', loss='soft_l1',  # Huber loss is more robust to outliers
                f_scale=5,                 # Scale parameter for loss function
                gtol=GTOL, xtol=XTOL, verbose=0
            )
        except Exception as e:
            print(f"Optimization failed for date {date}: {str(e)}")
            continue

        # Extract optimized parameters
        lat_f, lon_f, origin_time_offset_f, b_f = res.x

        # Calculate absolute origin time by adding the offset to reference time
        origin_time = ref_time + timedelta(seconds=origin_time_offset_f)

        # Calculate detailed residuals for analysis
        detailed_residuals = []
        for idx, (station_pos, sec) in enumerate(detection_secs):
            slat, slon = station_pos
            # Get travel time using final parameters
            travel_time, terr, dist = compute_enhanced_travel_time(
                lat_f, lon_f,
                slat, slon,
                b=b_f
            )

            # Predicted arrival time (seconds from ref_time)
            # origin time offset + travel time = predicted detection time
            pred_arrival_sec = origin_time_offset_f + travel_time

            # Residual = observed - predicted (in seconds)
            residual_sec = sec - pred_arrival_sec

            # Also calculate absolute times for human readability
            absolute_detection_time = ref_time + timedelta(seconds=sec)
            absolute_predicted_time = ref_time + timedelta(seconds=pred_arrival_sec)

            # Calculate station weight based on distance (inverse weighting)
            station_weight = 1 / (dist + 1e-5)  # Avoid division by zero

            detailed_residuals.append({
                'station_pos': station_pos,
                'observed_time_sec': sec,
                'absolute_detection_time': absolute_detection_time,
                'predicted_time_sec': pred_arrival_sec,
                'absolute_predicted_time': absolute_predicted_time,
                'residual_sec': residual_sec,
                'distance_m': dist,
                'travel_time_sec': travel_time,
                'estimated_error_sec': terr,
                'station_weight': station_weight,
            })

        # Calculate weighted RMS error
        squared_errs = np.array([r['residual_sec']**2 for r in detailed_residuals])
        weights = np.array([r['station_weight'] for r in detailed_residuals])
        norm_weights = weights / weights.sum()
        weighted_rms = np.sqrt(np.sum(squared_errs * norm_weights))

        # Calculate standard RMS for comparison
        standard_rms = np.sqrt(np.mean(squared_errs))

        # Store detailed optimization results for statistical analysis
        optimization_details = {
            'x': res.x.tolist(),            # Final parameters
            'cost': res.cost,               # Final cost function value
            'fun': res.fun.tolist(),        # Final residuals
            'nfev': res.nfev,               # Number of function evaluations
            'njev': res.njev,               # Number of Jacobian evaluations
            'status': res.status,           # Termination status
            'message': res.message,         # Termination message
            'success': res.success          # Boolean flag indicating success
        }

        validated.append({
            'station_positions': station_positions,
            'detection_times': detection_times,
            'source_point': (lat_f, lon_f),
            'origin_time': origin_time,
            'model_params': {'b': b_f},  # Save enhanced model parameters
            'weighted_rms_error': weighted_rms,
            'standard_rms_error': standard_rms,
            'num_stations': len(detection_secs),
            'optimization_success': res.success,
            'optimization_details': optimization_details,
            'detailed_residuals': detailed_residuals  # Add per-station residuals
        })

    return date, validated


if __name__ == "__main__":
    # Load input
    associations = np.load(ASSO_FILE, allow_pickle=True).item()
    items = list(associations.items())
    validated_associations = {}

    # Process in batches with checkpoints
    for i in range(0, len(items), BATCH_SIZE):
        batch = items[i:i+BATCH_SIZE]
        results = Parallel(n_jobs=N_JOBS, verbose=5)(
            delayed(process_date)(date, lst) for date, lst in batch
        )
        for date, val in results:
            validated_associations[date] = val

        # Checkpoint
        chkpt_path = os.path.join(
            OUTPUT_DIR,
            f"{OUTPUT_BASENAME}_partial_{i+BATCH_SIZE}.npy"
        )
        np.save(chkpt_path, validated_associations)
        print(f"✔ Checkpoint saved for first {i+BATCH_SIZE} dates → {chkpt_path}")

    # Final save
    final_path = os.path.join(OUTPUT_DIR, f"{OUTPUT_BASENAME}_final.npy")
    np.save(final_path, validated_associations)
    print(f"🎯 All done! Final results saved to {final_path}")

[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 tasks      | elapsed:    5.3s
[Parallel(n_jobs=7)]: Done  67 tasks      | elapsed:    5.6s
[Parallel(n_jobs=7)]: Done 500 out of 500 | elapsed:    8.5s finished
[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 tasks      | elapsed:    0.0s


✔ Checkpoint saved for first 500 dates → F:/Association/validated\s_-60-5,35-120,350,0.8,0.6_partial_500.npy


[Parallel(n_jobs=7)]: Done 102 tasks      | elapsed:    1.2s
[Parallel(n_jobs=7)]: Done 452 tasks      | elapsed:    4.4s
[Parallel(n_jobs=7)]: Done 500 out of 500 | elapsed:    5.0s finished
[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 tasks      | elapsed:    0.0s


✔ Checkpoint saved for first 1000 dates → F:/Association/validated\s_-60-5,35-120,350,0.8,0.6_partial_1000.npy


[Parallel(n_jobs=7)]: Done 102 tasks      | elapsed:    1.2s
[Parallel(n_jobs=7)]: Done 282 tasks      | elapsed:    3.7s
[Parallel(n_jobs=7)]: Done 500 out of 500 | elapsed:    7.4s finished
[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.


...

✔ Checkpoint saved for first 13000 dates → F:/Association/validated\s_-60-5,35-120,350,0.8,0.6_partial_13000.npy


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done 198 tasks      | elapsed:    0.9s
[Parallel(n_jobs=7)]: Done 500 out of 500 | elapsed:    2.9s finished


✔ Checkpoint saved for first 13500 dates → F:/Association/validated\s_-60-5,35-120,350,0.8,0.6_partial_13500.npy


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done  46 out of  75 | elapsed:    0.4s remaining:    0.2s
[Parallel(n_jobs=7)]: Done  62 out of  75 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=7)]: Done  75 out of  75 | elapsed:    0.4s finished


✔ Checkpoint saved for first 14000 dates → F:/Association/validated\s_-60-5,35-120,350,0.8,0.6_partial_14000.npy
🎯 All done! Final results saved to F:/Association/validated\s_-60-5,35-120,350,0.8,0.6_final.npy


In [13]:
validated_associations

{datetime.datetime(2018, 1, 22, 2, 39, 55, 581197): [],
 datetime.datetime(2018, 1, 22, 7, 56, 23, 534188): [],
 datetime.datetime(2018, 1, 22, 7, 45, 56, 931624): [],
 datetime.datetime(2018, 1, 20, 8, 22, 6, 29915): [],
 datetime.datetime(2018, 1, 22, 2, 33, 20, 521368): [],
 datetime.datetime(2018, 1, 22, 4, 13, 45, 735043): [],
 datetime.datetime(2018, 1, 22, 7, 22, 8, 700855): [],
 datetime.datetime(2018, 1, 22, 8, 3, 54, 923077): [],
 datetime.datetime(2018, 1, 21, 18, 22, 5, 568376): [],
 datetime.datetime(2018, 1, 22, 1, 18, 5, 111111): [],
 datetime.datetime(2018, 1, 22, 8, 19, 22, 34188): [],
 datetime.datetime(2018, 1, 21, 8, 35, 9, 683761): [],
 datetime.datetime(2018, 1, 21, 0, 4, 18, 722222): [],
 datetime.datetime(2018, 1, 20, 17, 36, 28, 316239): [],
 datetime.datetime(2018, 1, 22, 7, 36, 30, 987179): [],
 datetime.datetime(2018, 1, 21, 8, 24, 46, 632479): [],
 datetime.datetime(2018, 1, 22, 10, 44, 14, 735043): [],
 datetime.datetime(2018, 1, 21, 8, 22, 5, 495726): [],

In [8]:
1864+4980

6844